# Homework 5, Question 3 (40 points)
In this question, we explore the R.O.A.D. methodology. We will consider `spleen_mlopt_final_train.csv` and `spleen_mlopt_final_test.csv`, a subset of data on splenectomy that consists of 2,400 individuals. The features, treatment, and outcome variables are as follows:
- Features: `sex`, `age`, `sbp`, `pulserate`, `respiratoryrate`, `pulseoximetry`, `totalgcs`, `intubated`, `bmi`, etc.
- Treatment: `treatment`, which could either be "splenectomy" (spleen removal surgery, e.g. treatment) or "observation" (control - e.g. no surgery)
- Outcome: `o_mortality`, where 1 indicates patient death ("expiration") and 0 otherwise

*__Important:__* Please note to use the seed provided in all places (data splitting and any tree training), and do not change anything regarding the order/columns in the `spleen.csv` dataset, unless otherwise specified. 

In [ ]:
# Load packages
# If you need to install packages, please do not leave the output of installation in your homework submission.
using CSV, DataFrames, CategoricalArrays, Plots, Statistics, Random, StatsPlots, Gurobi, JuMP

seed = 42;

#### Read in the data and get the features, treatment, and outcome

In [ ]:
train = CSV.read("spleen_mlopt_final_train.csv", DataFrame)
test = CSV.read("spleen_mlopt_final_test.csv", DataFrame)

X_train = train[:, Not([:treatment, :o_mortality])]
t_train = train[:, :treatment]
y_train = train[:, :o_mortality]

X_test = test[:, Not([:treatment, :o_mortality])]
t_test = test[:, :treatment]
y_test = test[:, :o_mortality];

## Part 1
In this part, we will perform the first main step of R.O.A.D., which removes observed confounding by selecting a matched dataset from the original set of data. 

### Part 1(a) - 5 points

Using the *__control patients in the training dataset__*, train a `RandomForest` via `GridSearch` to predict mortality outcome; use cross-validation with 5 folds and criterion of AUC to finetune the hyperparameters amongst the following possible values: `max_depth` of [5], `minbucket` of [20, 50], and `num_trees` of [50, 100]. This model is trained to estimate mortality outcomes if patients did not receive treatment. 

Please answer the following questions.
- How good is our estimator? That is, what is the AUC and accuracy of our estimator on the control patients in the test set?
- Use this model to predict what the the mortality risk (e.g. the probability of mortality) would be if all the training set patients did not receive treatment. What is the mortality risk for the last training set patient (patient \#1920)? 

In [ ]:
idx_train_control = findall(val -> val == "observation", t_train)
X_train_control = X_train[idx_train_control, :]
t_train_control = t_train[idx_train_control]
y_train_control = y_train[idx_train_control]

idx_test_control = findall(val -> val == "observation", t_test)
X_test_control = X_train[idx_test_control, :]
t_test_control = t_train[idx_test_control]
y_test_control = y_train[idx_test_control];

In [ ]:
# Train RF on control training data
depths = [5]
minbuckets = [20, 50]
num_trees = [50, 100]

# TODO
# grid_rf = [TODO]
# IAI.fit_cv!([TODO])

# Get best model
lnr = IAI.get_learner(grid_rf)

In [ ]:
# Calculate AUC, accuracy for test set 
# println("Test AUC score: ", round([TODO], digits=4))
# println("Test Accuracy: ", round([TODO], digits=4))

In [ ]:
# print("Mortality risk of last training patient: ", [TODO])

### Part 1(b) - 2.5 points

We now divide the training test patients into 5 equally-sized buckets, in order of increasing predicted mortality risk. How many patients received treatment in Bucket 5? You should fill in the TODO section to print this.

In [ ]:
# Sort and bucket
y_proba = IAI.predict_proba(lnr, X_train)[:, 2]
train[!, "y_proba"] = y_proba
train = sort!(train, :y_proba);

groups = []
num_groups = 5
num_per_group = Integer(ceil(size(train)[1] / num_groups))

for idx in 1:num_groups
    start_idx = (idx-1)*num_per_group+1
    if idx==num_groups
        end_idx = size(train)[1]
    else
        end_idx = idx*num_per_group
    end
    println(start_idx)
    println(end_idx)
    subdf = train[start_idx:end_idx, :]
    push!(groups, subdf)
    frequencies = sort!(combine(groupby(subdf, :treatment), nrow => :frequency), :treatment)

    # [TODO]
end

### Part 1(c) - 2.5 points

For each bucket $k\in 1,...,5$, we compute a corresponding distance matrix $\textbf{D}_k$, such that $\textbf{D}_k[i,j]$ is the squared Euclidean norm between the $i$-th treatment group individual and the $j$-th control group individual. The distance should be computed on three features: [`sex`, `age`, `bmi`]. Make sure to normalize the non-binary features before computing the distance, by subtracting the full training mean and dividing by the full training range, such that they fall between 0 and 1. Formally, let $x_{i,s}$ be the value of the feature $s$ for individual $i$; then the normalized value $\hat{x}_{i,s}$ is defined:
        $$\hat{x}_{i,s} = \frac{x - \min_{j\in\mathcal{T}}(x)}{\max_{j\in\mathcal{T}}(x) - \min_{j\in\mathcal{T}}(x)}, \quad \forall i\in\mathcal{T}, \forall s,$$
 where $\mathcal{T}$ is the set of training datapoints.
        
What is the dimension of $\mathbf{D}_{5}$ and what is the distance between treatment individual 10 and control individual 12 in the 5th bucket?

In [ ]:
function compute_dist(row1, row2, features)
    dist = 0
    for f in features
        dist += (row1[f] - row2[f])^2
    end
    return dist
end

function compute_dist_matrix(df_treatment1, df_treatment2, features)
    n1 = size(df_treatment1)[1]
    n2 = size(df_treatment2)[1]
    distances = zeros(n1, n2)
    for i in 1:n1
        for j in 1:n2
            distances[i,j] = compute_dist(df_treatment1[i, :], df_treatment2[j, :], features)
        end
    end
    return distances
end

min_age = minimum(train[:, "age"])
max_age = maximum(train[:, "age"])
min_bmi = minimum(train[:, "bmi"])
max_bmi = maximum(train[:, "bmi"])

matching_features = ["sex", "age", "bmi"]
group_dist = []
std_gs = []
for g in groups
    std_g = copy(g)[:, vcat(matching_features, ["treatment"])]
    # [TODO: normalize non-binary features - you may use the min/max values computed above]
    # [TODO: call the compute_dist_matrix function with the appropriate arguments to compute the distance matrix for each group]
    # dist_matrix = ...
    
    push!(group_dist, dist_matrix)
end

In [ ]:
# println("Shape of bucket 5: ", [TODO])
# println("Distance b/w treatment 10 and control 12 in bucket 5: ", [TODO])

### Part 1(d) - 7 points

For each bucket $k\in 1,...,5$, implement and run the matching algorithm from lecture:
        $$\begin{aligned}
            \min_{\boldsymbol{z}} & \quad \sum_{i\in\mathcal{S}_1^k}\sum_{j\in\mathcal{S}_0^k} z_{i,j} \textbf{D}_k[i,j] \\
            \text{s.t.}& \quad \sum_{j\in\mathcal{S}_0^k}z_{i,j} = 1, \quad \forall i \in \mathcal{S}_1^k \\
            & \quad \sum_{i\in\mathcal{S}_1^k}z_{i,j} \leq 1, \quad \forall j \in \mathcal{S}_0^k \\
            &\quad z_{i,j}\in\{0,1\} \quad \forall i \in\mathcal{S}_1^k,~j\in\mathcal{S}_0^k,
        \end{aligned}$$
where ${S}_0^k, {S}_1^k$ is the set of control patients and the set of treatment patients in bucket $k$ respectively. Report the control patients that are matched in the first bucket. You may report them by index.

In [ ]:
function matching(dist_matrix)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)

    # [TODO]
    
    optimize!(m);
    assignment = value.(z)
    return objective_value(m), assignment
end

In [ ]:
data_list = []
for i in 1:length(groups)
    g_dist = group_dist[i]
    obj, assignment = matching(g_dist)
    g = groups[i]
    control = filter(row -> row.treatment == "observation", g)
    treatment = filter(row -> row.treatment == "splenectomy", g)
    
    control_matched_idx = [idx[1] for idx in findall(val -> val == 1, assignment)]
    control_matched = control[control_matched_idx, :]
    push!(data_list, treatment)
    push!(data_list, control_matched)

    # [TODO: print the control patients that are matched in the first bucket]
end

## Part 2

We now perform the second main step of R.O.A.D., which aims to remove unobserved confounding. Recall that the first step of R.O.A.D, implemented in Part 2, produces matched pairs of treatment and control patients. We can combine these matched pairs across buckets to get a "cleaner'" subset of the full dataset ("removed" observed confounding). We will call this the matched dataset. We now train two estimators -- one for the control group and one for the treatment group -- to perform counterfactual estimation, while finetuning a  parameter $\rho$ to remove the unobserved confounding.

We begin by combine the matched pairs across buckets to produce the matched dataset.

In [ ]:
matched_data = reduce(vcat, data_list)
first(matched_data, 5)

### Part 2(a) - 2.5 points

On the control patients in the matched dataset, train a `RandomForest` via `GridSearch` to predict the outcome; use cross-validation to finetune the hyperparameters amongst the same possible values as in Part 1(a). This is training a *__control__* model; we denote the predicted mortality risk from this model for patient $\boldsymbol{x}_i$ as $h_{t=0}(\boldsymbol{x}_i)$. 
     
How good is our estimator? That is, what is the AUC and accuracy of our estimator on the control patients in the test set? 

In [ ]:
matched_data_control = filter(row -> row.treatment == "observation", matched_data)
matched_data_treatment = filter(row -> row.treatment == "splenectomy", matched_data);

In [ ]:
# Train direct estimator for control

# grid_rf = [TODO]

# IAI.fit_cv!([TODO])

# Get best model
lnr_control = IAI.get_learner(grid_rf)

In [ ]:
# Calculate AUC, accuracy for test set 
# println("Test AUC score: ", round([TODO], digits=4))
# println("Test Accuracy: ", round([TODO], digits=4))

### Part 2(b) - 2.5 points

Now train a `RandomForest` via `GridSearch` on the treatment patients in the matched dataset to predict the outcome; use cross-validation to finetune the hyperparameters amongst the same possible values as in Part 2(a). This is training a *__treatment__* model; we denote the predicted mortality risk from this model for patient $\boldsymbol{x}_i$ as $h_{t=1}(\boldsymbol{x}_i)$. 
        
How good is our estimator? That is, what is the AUC and accuracy of our estimator on the control patients in the test set? 

In [ ]:
# Train direct estimator for treatment

# grid_rf = IAI.GridSearch([TODO])

# IAI.fit_cv!([TODO])

# Get best model
lnr_treatment = IAI.get_learner(grid_rf);

In [ ]:
# Calculate AUC, accuracy for test set 
# println("Test AUC score: ", round([TODO], digits=4))
# println("Test Accuracy: ", round([TODO], digits=4))

### Part 2(c) - 3 points

Use both the control model and the treatment model to predict the mortality risk on the matched dataset. What do you notice about the outcomes of the control model and the treatment model? In particular, answer this by evaluating and reporting the following quantity on the matched dataset: $$\hat{w}_{t=1} - \hat{w}_{t=0},\quad\text{where }\hat{w}_{t=1} = \frac{1}{n_s}\sum_{i=1}^{n_s} h_{t=1}(\boldsymbol{x}_i), ~\hat{w}_{t=0} = \frac{1}{n_s}\sum_{i=1}^{n_s}h_{t=0}(\boldsymbol{x}_i),$$
where $n_s$ is the number of patients in our matched dataset. 

What does this mean, are there any issues, and what should we do about it, if anything? (Please answer this in a __markdown__ cell below

In [ ]:
X_matched = matched_data[:, Not([:treatment, :o_mortality, :y_proba])]

# [TODO]

[Space to answer the question]

### Part 2(d) - 5 points

Implement your suggestion in Part 2(c). 

*__Hint:__* If your suggestion includes tuning the parameter $\rho$, then finetune $\rho$ across the following possible values: [1, 1.5, 2, 2.5]. For each $\rho$ value, train a corresponding *__treatment__* model, e.g. repeat Part 2(c), *__but in a sample-weighted manner__* such that the surviving (non-expired) patients have weight $\rho$ and expired patients have weight $1$ (see [documentation](https://docs.interpretable.ai/stable/IAIBase/data/#Sample-Weights)). For each model trained in this manner with the corresponding $\rho$, evaluate the corresponding quantity $\hat{w}_{t=1} - \hat{w}_{t=0}$. Select the $\rho$ and corresponding treatment model such that this quantity is closest \textit{in magnitude} to zero. Report your selected $\rho$.


In [ ]:
best_rho = nothing
best_diff = nothing
best_lnr_treatment = nothing

for rho in [1, 1.5, 2, 2.5]
    println("Running for rho ", rho)
    # grid_rf = [TODO]
    
    # IAI.fit_cv!([TODO])
    
    # Get best model
    lnr_treatment = IAI.get_learner(grid_rf)
    
    # Display selected model's parameters
    # diff = [TODO: w_1 - w_0 quantity]
    
    if best_rho == nothing || abs(diff) < abs(best_diff)
        best_rho = rho
        best_diff = diff
        best_lnr_treatment = lnr_treatment
    end
end
println("******************")
println("Best rho ", best_rho)

### Part 2(e) - 4 points

We use the learner for the control model and the learner for the treatment model to estimate outcomes for the matched data and organize this into a `DataFrame` where the rows are the individuals and the columns are treatment/no treatment. This will serve as our rewards matrix.

Using the matched data and this rewards matrix, train an `Optimal Policy Tree` via `GridSearch` and use cross-validation to finetune the hyperparameters amongst the following possible values: `max_depth` of [4, 5] and `minbucket` of [20, 50]. Provide a screenshot of the tree in your writeup (or make sure it is visible in your code). What is the sensitivity and specificity on the *__test set__*?  Recall the definition of sensitivity and specificity in R.O.A.D.:
- Sensitivity = out of the historically-untreated patients that expired, what fraction were prescribed treatment by the model
- Specificity = out of the historically-untreated patients that did not expire (survived), what fraction were not prescribed treatment by the model

How can we interpret the sensitivity and specificity of our R.O.A.D. model and what is a critical assumption about the treatment that we use when training and evaluating our model? Please answer this question in the provided *__markdown__* cell.

In [ ]:
# rewards matrix

rewards_c_matched = IAI.predict_proba(lnr_control, X_matched)[:, 2];
rewards_t_matched = IAI.predict_proba(best_lnr_treatment, X_matched)[:, 2]
rewards_matched = DataFrame(Dict("observation" => rewards_c_matched, "splenectomy" => rewards_t_matched))
first(rewards_matched, 5)

In [ ]:
# grid_opt_road = [TODO]

# IAI.fit_cv!([TODO])

In [ ]:
IAI.show_in_browser(grid_opt_road)

In [ ]:
# sensitivity = [TODO]
# specificity = [TODO]

# println("Sensitivity: ", sensitivity)
# println("Specificity: ", specificity)

[Space to answer the question]

### Part 2(f) - 4 points

We will now compare the R.O.A.D. model with an OPT trained on the unadulterated training data. Using the *__original__* data, train a *__doubly-robust__* reward estimator (see [documentation](https://docs.interpretable.ai/stable/OptimalTrees/quickstart/policy_categorical/)) on the training set to get a rewards matrix. Use the original data and this new rewards matrix to train an `Optimal Policy Tree` via `GridSearch` and use cross-validation to finetune the hyperparameters amongst the same possible values as 3(e). Provide a screenshot of the tree in your writeup (or make sure it is visible in your code). What is the sensitivity and specificity?

How doees this tree differ from the tree trained via the R.O.A.D. methodology? How does the sensitivity/specificity differ? What does this mean? Please be concise in your answer (<=5 sentences) and use the provided *__markdown__* space.

In [ ]:
# reward_lnr = [TODO]
# train_pred, train_reward_score = IAI.fit_predict!([TODO])
# rewards_train = train_pred[:reward];

In [ ]:
# grid_opt_regular = [TODO]
# IAI.fit_cv!([TODO])

In [ ]:
# IAI.show_in_browser(grid_opt_regular)

In [ ]:
# sensitivity = [TODO]
# specificity = [TODO]

# println(sensitivity)
# println(specificity)

[Space to answer the question]